In [ ]:
# import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# imoprt the dataset
df = pd.read_csv('../input/sales-of-shampoo/sales-of-shampoo-over-a-three-ye.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df = pd.read_csv('../input/sales-of-shampoo/sales-of-shampoo-over-a-three-ye.csv', parse_dates=True,index_col=[0],squeeze=True)
df.head()

In [ ]:
type(df)

In [ ]:
# we will visaulise our data
df.plot()

In [ ]:
df.plot(style = 'k.')

we will try to smooth the data as we can see that our data is varied alot, we will smooth by taking average.

In [ ]:
df_ma = df.rolling(window=10).mean()
df_ma

Here we performed the rolling operation on the dataset as we needed to smooth the data,in rolling we considered 10 rows in order from 1st to 10th and took the average of it and printed the value of it at 10 the position. The data above the 10 rows will be NaN as it does not have adequate rows to print the average of the 10 rows.

In [ ]:
# we will now plot the smoothed dataframe.
df_ma.plot()

In [ ]:
# we will now compare the smoothed data to the original data.
df.plot()
df_ma.plot()

#### Now we will create a Naive(baseline) model to get the error of the model.

In [ ]:
df

In [ ]:
df_naive = pd.concat([df,df.shift(1)],axis=1)
df_naive

In [ ]:
df_naive.dropna(inplace=True)
df_naive.columns = ['Actual Data','Shifted Data']
df_naive 

We can see that our data is now prepared.

In [ ]:
# we will get the mean_sqaured_error
from sklearn.metrics import mean_squared_error

In [ ]:
error_1 = mean_squared_error(df_naive['Actual Data'],df_naive['Shifted Data'])
error_1

In [ ]:
error_1 = np.sqrt(error_1)
error_1

Here we got the error of our naive model around 109. Now we will create ARIMA model , we need to make sure that the error margin of our model should be less than the error margin of the naive model, then only our model will be said as a good model.

## ARIMA( Auto Regressive Integrated Moving Average)

* Auto Regressive = p value
* Integrated = d value
* Moving Average = q value

We can see that our data is not stationary so we can take the value of d as 1.

We will find the p and q values with plot_acf and plot_pacf.

#### arima(p,d,q)
* If want an (AR)autoregressive model we will take the value as arima(2,0,0) any value of p
* If want an (MV)Moving average model we will take the value as arima(0,0,2) any value of q
* If want an whole ARIMA model we will take the value as arima(2,1,2) any values for all the three.

In [ ]:
# acf and pacf
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
# Here we are considering the original dataframe, our original dataframe consisted NaN value in the last row so we dropped it.
df.dropna(inplace=True)

In [ ]:
# plot_acf will give us the q value(Autocorrelation)

plot_acf(df)

Here We can see that first point has correlation of 1 , so we count from the next value untill the value goes into the critical range. We can see that there are 3 points which are not below the critical range.
So we take the value of q = 3.

In [ ]:
# plot_pacf will give us the q value(Partial Autocorrelation)

plot_pacf(df)

Here We can see that first point has correlation of 1 , so we count from the next value untill the value goes into the critical range. We can see that there are 3 points which are not below the critical range.
So we take the value of q = 2.

In [ ]:
# p = 2
# d = 1
# q = 3

In [ ]:
# we will now create our ARIMA model.
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
# we will now create a training and testing data 
df_train = df[:26]
df_test = df[26:]
df_train.shape,df_test.shape

In [ ]:
model = ARIMA(df_train, order=(3,1,2))

In [ ]:
model_fit = model.fit()

In [ ]:
model_fit.aic

we can try different values of the ARIMA model and try to find the aic value, the model who has the least aic value will be the best model.

In [ ]:
model_forecast = model_fit.forecast(steps=10)[0]

In [ ]:
error2 = np.sqrt(mean_squared_error(df_test,model_forecast))
error2

From this we can say that our ARIMA model is not forecasting better than our naive model.

### To find which model works best with altering p,d,q values.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
p_values = range(0,5)
d_values = range(0,3)
q_values = range(0,5)

In [ ]:
for p in p_values:
    for d in d_values:
        for q in q_values:
            order = (p,d,q)
            train,test = df[0:26], df[26:]
            predictions = []
            for i in range(len(test)):
                try:
                    model = ARIMA(train, order)
                    model_fit = model.fit(disp=0)
                    pred_y = model_fit.forecast()[0]
                    predictions.append(pred_y)
                    error = np.sqrt(mean_squared_error(test,predictions))
                    print('Arima%s RMSE = %.2f'% (order,error))
                except:
                    continue

We can see that no order combination works better in reducing the error for our model than the error of our naive model.